# 🚀Analise da tabela Stage Area

### 📌 Passo 1: Configurar a Conexão com o Banco

In [2]:
!pip install sqlalchemy
import pandas as pd
from sqlalchemy import create_engine
import os

# 🔹 Configurar conexão com o banco
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

# 🔹 Carregar dados da Staging Area
df_stg = pd.read_sql("SELECT * FROM stg_acidentes", con=engine)

# 🔍 Exibir as primeiras linhas para análise
df_stg.head()


,id,concessionaria,data,horario,n_da_ocorrencia,tipo_de_ocorrencia,km,trecho,sentido,tipo_de_acidente,...,outros,tracao_animal,transporte_de_cargas_especiais,trator_maquinas,utilitarios,ilesos,levemente_feridos,moderadamente_feridos,gravemente_feridos,mortos
0,256630c4-69b2-4d09-9481-4659040fb051,AUTOPISTA_FERNAO_DIAS,01/01/2010,03:02:00,20,sem vítima,506.5,BR-381/MG,Sul,"Choque - Defensa, barreira ou submarino""""",...,0,0,0,0,0,1,0,0,0,0
1,d5924260-52cf-4a95-981e-f97e652710dc,AUTOPISTA_FERNAO_DIAS,01/01/2010,06:16:00,39,sem vítima,767,BR-381/MG,Sul,Saida de Pista,...,0,0,0,0,0,2,0,0,0,0
2,955e1026-8088-4555-896c-47366a3b62df,AUTOPISTA_FERNAO_DIAS,01/01/2010,06:37:00,42,com vítima,567,BR-381/MG,Sul,Choque - Suporte de Sinalização,...,0,0,0,0,0,0,1,0,0,0
3,1959f647-2d87-4181-9ac9-1032b3731ea7,AUTOPISTA_FERNAO_DIAS,01/01/2010,06:49:00,44,sem vítima,492,BR-381/MG,Norte,Saida de Pista,...,0,0,0,0,0,1,0,0,0,0
4,e02e1b60-dcb8-41ae-8814-9c4a1bbb2437,AUTOPISTA_FERNAO_DIAS,01/01/2010,07:11:00,48,sem vítima,76.5,BR-381/SP,Sul,"Choque - Defensa, barreira ou submarino""""",...,0,0,0,0,0,1,0,0,0,0


### 📌 Passo 1: Criar as Dimensões

In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Criar função para salvar dimensões no banco
def save_dimension(df, table_name, engine):
    """Salva uma dimensão no banco e adiciona surrogate key (sk)"""
    df["sk"] = range(1, len(df) + 1)  # Criar Surrogate Key
    df.to_sql(table_name, con=engine, if_exists="replace", index=False)
    print(f"Dimensão {table_name} carregada com sucesso.")

# 1️⃣ Criar Dimensão Situação do Acidente
dim_situacao_acidente = pd.DataFrame({
    "tipo": ["ilesos", "levemente_feridos", "moderadamente_feridos", "gravemente_feridos", "mortos"]
})
save_dimension(dim_situacao_acidente, "dim_situacao_acidente", engine)

# 2️⃣ Criar Dimensão Tipo de Veículo
dim_tipo_veiculo = pd.DataFrame({
    "tipo_veiculo": ["automovel", "bicicleta", "caminhao", "moto", "onibus",
                     "outros", "tracao_animal", "transporte_de_cargas_especiais",
                     "trator_maquinas", "utilitarios"]
})
save_dimension(dim_tipo_veiculo, "dim_tipo_veiculo", engine)

# 3️⃣ Criar Dimensão Tipo de Ocorrência
dim_tipo_ocorrencia = df_stg[["tipo_de_ocorrencia"]].drop_duplicates().dropna().reset_index(drop=True)
save_dimension(dim_tipo_ocorrencia, "dim_tipo_ocorrencia", engine)

# 4️⃣ Criar Dimensão Sentido
dim_sentido = df_stg[["sentido"]].drop_duplicates().dropna().reset_index(drop=True)
save_dimension(dim_sentido, "dim_sentido", engine)

# 5️⃣ Criar Dimensão Rodovia
dim_rodovia = df_stg[["concessionaria"]].drop_duplicates().dropna().reset_index(drop=True)
save_dimension(dim_rodovia, "dim_rodovia", engine)

# 6️⃣ Criar Dimensão Trecho
dim_trecho = df_stg[["trecho"]].drop_duplicates().dropna().reset_index(drop=True)
save_dimension(dim_trecho, "dim_trecho", engine)


Dimensão dim_situacao_acidente carregada com sucesso.
Dimensão dim_tipo_veiculo carregada com sucesso.
Dimensão dim_tipo_ocorrencia carregada com sucesso.
Dimensão dim_sentido carregada com sucesso.
Dimensão dim_rodovia carregada com sucesso.
Dimensão dim_trecho carregada com sucesso.


### 📌 Passo 2: Criar a Tabela Fato

In [10]:
# 🔹 Carregar dados da Staging Area
df_stg = pd.read_sql("SELECT * FROM stg_acidentes", con=engine)

# 🔹 Função para mapear valores das dimensões para suas surrogate keys (sk)
def map_sk(df, dim_table, key_column, sk_column, engine):
    """Mapeia surrogate keys das dimensões na fato"""
    dim_df = pd.read_sql(f"SELECT sk AS {sk_column}, {key_column} FROM {dim_table}", con=engine)
    df = df.merge(dim_df, how="left", left_on=key_column, right_on=key_column).drop(columns=[key_column])
    return df

# 🔹 Lista de colunas de veículos e situações
veiculos_cols = ["automovel", "bicicleta", "moto", "caminhao", "onibus", "outros", 
                 "tracao_animal", "transporte_de_cargas_especiais", "trator_maquinas", "utilitarios"]

situacao_cols = ["ilesos", "levemente_feridos", "moderadamente_feridos", "gravemente_feridos", "mortos"]

# 🔹 Criar tabela fato long para veículos
df_fato_veiculos = df_stg.melt(id_vars=["id", "data", "horario", "km", "tipo_de_ocorrencia", 
                                         "sentido", "trecho", "concessionaria", "tipo_de_acidente"],
                               value_vars=veiculos_cols, var_name="tipo_veiculo", value_name="quantidade_veiculos")

# 🔹 Criar tabela fato long para situações das vítimas
df_fato_situacoes = df_stg.melt(id_vars=["id", "data", "horario", "km", "tipo_de_ocorrencia", 
                                          "sentido", "trecho", "concessionaria", "tipo_de_acidente"],
                                value_vars=situacao_cols, var_name="tipo", value_name="quantidade_vitimas")

# 🔹 Mapear surrogate keys nas tabelas fato para veículos
df_fato_veiculos = map_sk(df_fato_veiculos, "dim_tipo_ocorrencia", "tipo_de_ocorrencia", "sk_tipo_ocorrencia", engine)
df_fato_veiculos = map_sk(df_fato_veiculos, "dim_sentido", "sentido", "sk_sentido", engine)
df_fato_veiculos = map_sk(df_fato_veiculos, "dim_trecho", "trecho", "sk_trecho", engine)
df_fato_veiculos = map_sk(df_fato_veiculos, "dim_rodovia", "concessionaria", "sk_rodovia", engine)
df_fato_veiculos = map_sk(df_fato_veiculos, "dim_tipo_veiculo", "tipo_veiculo", "sk_tipo_veiculo", engine)

# 🔹 Mapear surrogate keys nas tabelas fato para situações das vítimas
df_fato_situacoes = map_sk(df_fato_situacoes, "dim_tipo_ocorrencia", "tipo_de_ocorrencia", "sk_tipo_ocorrencia", engine)
df_fato_situacoes = map_sk(df_fato_situacoes, "dim_sentido", "sentido", "sk_sentido", engine)
df_fato_situacoes = map_sk(df_fato_situacoes, "dim_trecho", "trecho", "sk_trecho", engine)
df_fato_situacoes = map_sk(df_fato_situacoes, "dim_rodovia", "concessionaria", "sk_rodovia", engine)
df_fato_situacoes = map_sk(df_fato_situacoes, "dim_situacao_acidente", "tipo", "sk_situacao_acidente", engine)

# 🔹 Remover registros com quantidade 0 para evitar armazenar dados irrelevantes
# 🔹 Converter colunas para tipo numérico (caso tenham sido carregadas como string)
df_fato_veiculos["quantidade_veiculos"] = pd.to_numeric(df_fato_veiculos["quantidade_veiculos"], errors="coerce").fillna(0).astype(int)
df_fato_situacoes["quantidade_vitimas"] = pd.to_numeric(df_fato_situacoes["quantidade_vitimas"], errors="coerce").fillna(0).astype(int)

# 🔹 Agora podemos remover registros com quantidade 0
df_fato_veiculos = df_fato_veiculos[df_fato_veiculos["quantidade_veiculos"] > 0]
df_fato_situacoes = df_fato_situacoes[df_fato_situacoes["quantidade_vitimas"] > 0]


# 🔹 Concatenar ambas as tabelas fato
df_fato_final = pd.concat([df_fato_veiculos, df_fato_situacoes], ignore_index=True)

# 🔹 Adicionar surrogate key à tabela fato
df_fato_final["sk_acidente"] = range(1, len(df_fato_final) + 1)

# 🔹 Salvar a tabela fato no banco
df_fato_final.to_sql("fato_acidentes", con=engine, if_exists="replace", index=False)
print("Tabela fato_acidentes carregada com sucesso.")
